In [ ]:
# 라이브러리 설치 (Python version 3.13.3)
%pip install -r requirements.txt

In [1]:
import os
import json
import requests
import warnings

warnings.filterwarnings("ignore")

from collections import Counter
from dotenv import load_dotenv

from openai import OpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient, models

load_dotenv()
api_key = os.getenv("UPSTAGE_API_KEY")

## 1. Indexing

## Load

### Document Parse로 PDF 파싱하기

In [2]:
# 사용할 문서를 작성
filename = "./data/전자서명법(법률)(제18479호)(20221020).pdf"

# API 호출
url = "https://api.upstage.ai/v1/document-digitization"
headers = {"Authorization": f"Bearer {api_key}"}
files = {"document": open(filename, "rb")}
data = {
    "output_formats": "['html', 'markdown', 'text']",  # 출력 형식을 지정할 수 있음
    "ocr": "force",
    "base64_encoding": "['figure']",
    "model": "document-parse",
}
dp_result = requests.post(url, headers=headers, files=files, data=data)

# 결과를 json 파일로 저장
with open("./dp_result.json", "w", encoding="utf-8") as f:
    json.dump(dp_result.json(), f, ensure_ascii=False, indent=2)

In [3]:
# json 파일 로드
with open("./dp_result.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 파싱 결과 확인
categories = [elem["category"] for elem in data["elements"]]
print("[ Category 분포 확인 ]")
for category, count in Counter(categories).most_common():
    print(f"- {category}: {count}")

print("\n[ 파싱 결과 일부 확인 ]")
for elem in data["elements"][3:6]:
    print(f"ID {elem['id']}")
    print(f"- Category: {elem['category']}")
    print(f"- Content: {elem['content']['text'][:50]} ...")
    print(f"- Page: {elem['page']}\n")

[ Category 분포 확인 ]
- paragraph: 42
- list: 18
- footer: 18
- header: 6
- heading1: 2

[ 파싱 결과 일부 확인 ]
ID 3
- Category: paragraph
- Content: 제1조(목적) 이 법은 전자문서의 안전성과 신뢰성을 확보하고 그 이용을 활성화하기 위하여  ...
- Page: 1

ID 4
- Category: heading1
- Content: 제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. ...
- Page: 1

ID 5
- Category: list
- Content: 1. "전자문서"란 정보처리시스템에 의하여 전자적 형태로 작성되어 송신 또는 수신되거나 저 ...
- Page: 1



### 데이터 전처리

In [4]:
page_num = 0
category_name = "footer"  # paragraph, heading1, list, header, footer
print(f"[ 문서 구조 확인: {category_name} ]")
for elem in data["elements"]:
    if elem["page"] != page_num:
        page_num = elem["page"]
        print(f"\n=========== Page {page_num} ==========\n")

    if elem["category"] == category_name:
        print(elem["content"]["text"])

[ 문서 구조 확인: footer ]

=========== Page 1 ==========

법제처
1
국가법령정보센터

=========== Page 2 ==========

법제처
2
국가법령정보센터

=========== Page 3 ==========

법제처
3
국가법령정보센터

=========== Page 4 ==========

법제처
4
국가법령정보센터

=========== Page 5 ==========

법제처
5
국가법령정보 센터

=========== Page 6 ==========

법제처
6
국가법령정보센터


In [5]:
texts = []
for elem in data["elements"]:
    category = elem["category"]
    text = elem["content"]["text"]
    # 필수 내용이 담긴 카테고리만 선택
    if category in ["paragraph", "list", "heading1"]:
        texts.append(text)
texts = "\n".join(texts)

print(f"- 전처리 전 텍스트 길이: {len(data["content"]["text"])}")
print(f"- 전처리 후 텍스트 길이: {len(texts)}")
print(f"\n- 첫 200자:\n{texts[:200]} ...")

- 전처리 전 텍스트 길이: 8796
- 전처리 후 텍스트 길이: 8669

- 첫 200자:
전자서명법
[시행 2022. 10. 20.] [법률 제18479호, 2021. 10. 19., 일부개정]
과학기술정보통신부 (정보보호기획과) 044-202-6445, 6447
제1조(목적) 이 법은 전자문서의 안전성과 신뢰성을 확보하고 그 이용을 활성화하기 위하여 전자서명에 관한 기본적인
사항을 정함으로써 국가와 사회의 정보화를 촉진하고 국민생활의 편익을  ...


In [6]:
# 제1조(목적) 이전 부분 제거
start_pos = texts.find("제1조(목적)")
if start_pos != -1:
    cleaned_texts = texts[start_pos:]
else:
    cleaned_texts = texts
print(f"- 첫 200자:\n{cleaned_texts[:200]} ...")

- 첫 200자:
제1조(목적) 이 법은 전자문서의 안전성과 신뢰성을 확보하고 그 이용을 활성화하기 위하여 전자서명에 관한 기본적인
사항을 정함으로써 국가와 사회의 정보화를 촉진하고 국민생활의 편익을 증진함을 목적으로 한다.
제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다.
1. "전자문서"란 정보처리시스템에 의하여 전자적 형태로 작성되어 송신 또는 수신되거나 저 ...


## Split

In [7]:
# 문서를 1000자 단위로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 각 청크의 최대 크기
    chunk_overlap=200,  # 청크 간 겹치는 부분 (맥락 유지)
)
split_texts = text_splitter.split_text(cleaned_texts)
print(f"총 {len(split_texts)}개의 청크로 분할 완료")

총 11개의 청크로 분할 완료


In [8]:
documents = []
for content in split_texts:
    document = {
        "content": content,
        "metadata": {
            "filename": "전자서명법",
            "enforcement_date": "2022-10-20",
            "law_id": 18479,
        },
    }
    documents.append(document)

print("[ 첫번째 Document 확인 ]\n")
print(f'- Content:\n{documents[0]["content"][:200]} ...')
print(
    f'\n- Metadata:\n{json.dumps(documents[0]["metadata"], ensure_ascii=False, indent=2)}'
)

[ 첫번째 Document 확인 ]

- Content:
제1조(목적) 이 법은 전자문서의 안전성과 신뢰성을 확보하고 그 이용을 활성화하기 위하여 전자서명에 관한 기본적인
사항을 정함으로써 국가와 사회의 정보화를 촉진하고 국민생활의 편익을 증진함을 목적으로 한다.
제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다.
1. "전자문서"란 정보처리시스템에 의하여 전자적 형태로 작성되어 송신 또는 수신되거나 저 ...

- Metadata:
{
  "filename": "전자서명법",
  "enforcement_date": "2022-10-20",
  "law_id": 18479
}


## Embedding & Store

In [9]:
# 임베딩 모델 불러오기
embedding_model = SentenceTransformer("Qwen/Qwen3-Embedding-4B")

Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.49s/it]


In [10]:
qdrant_path = "./qdrant"  # 로컬에 Qdrant 결과를 저장
collection_name = "law"  # 문서가 저장될 그룹의 이름

client = QdrantClient(path=qdrant_path)

# collection 생성
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=embedding_model.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE,  # 코사인 유사도 사용
    ),
)

True

In [11]:
all_points = []
for point_id, doc in enumerate(documents):
    # 텍스트를 벡터로 변환
    vector = embedding_model.encode(doc["content"]).tolist()

    # Qdrant에 저장할 point 생성
    point = models.PointStruct(
        id=point_id,
        vector=vector,  # 변환된 벡터
        payload={**doc},  # 벡터로 변환되지 않은 원본 텍스트와 메타데이터를 저장
    )
    all_points.append(point)

# 모든 벡터를 Qdrant에 업로드
client.upload_points(
    collection_name=collection_name,
    points=all_points,
)

## 2. Retrieval

In [12]:
user_query = "전자서명의 발전을 위한 시책에는 어떤 것들이 있어?"

# 질문을 벡터로 변환
query_vector = embedding_model.encode(user_query).tolist()

In [13]:
hits = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=3,  # 상위 3개로 설정
)

print("[ 검색 결과 (상위 3개) ]")
for i, hit in enumerate(hits):
    print(f"\n{i}번째 결과 (유사도: {hit.score:.4f})")
    print(f"Content: {hit.payload['content'][:250]}...")
    print("-" * 50)

[ 검색 결과 (상위 3개) ]

0번째 결과 (유사도: 0.7321)
Content: 제3조(전자서명의 효력) ① 전자서명은 전자적 형태라는 이유만으로 서명, 서명날인 또는 기명날인으로서의 효력이 부
인되지 아니한다.
② 법령의 규정 또는 당사자 간의 약정에 따라 서명, 서명날인 또는 기명날인의 방식으로 전자서명을 선택한 경우
그 전자서명은 서명, 서명날인 또는 기명날인으로서의 효력을 가진다.
제4조(전자서명의 발전을 위한 시책 수립) 정부는 전자서명의 안전성, 신뢰성 및 전자서명수단의 다양성을 확보하고 그
이용을 활성화하는 등...
--------------------------------------------------

1번째 결과 (유사도: 0.5466)
Content: 6. 그 밖에 전자서명의 이용 촉진을 위하여 필요한 사항
제6조(다양한 전자서명수단의 이용 활성화) ① 국가는 생체인증, 블록체인 등 다양한 전자서명수단의 이용 활성화를 위
하여 노력하여야 한다.
② 국가는 법률, 국회규칙, 대법원규칙, 헌법재판소규칙, 중앙선거관리위원회규칙, 대통령령 또는 감사원규칙에서
전자서명수단을 특정한 경우를 제외하고는 특정한 전자서명수단만을 이용하도록 제한하여서는 아니 된다.
제7조(전자서명인증업무 운영기준 등) ① 과...
--------------------------------------------------

2번째 결과 (유사도: 0.5346)
Content: 제1조(목적) 이 법은 전자문서의 안전성과 신뢰성을 확보하고 그 이용을 활성화하기 위하여 전자서명에 관한 기본적인
사항을 정함으로써 국가와 사회의 정보화를 촉진하고 국민생활의 편익을 증진함을 목적으로 한다.
제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다.
1. "전자문서"란 정보처리시스템에 의하여 전자적 형태로 작성되어 송신 또는 수신되거나 저장된 정보를 말한다.
2. "전자서명"이란 다음 각 목의 사항을 나타내는 데 이용하기 위하...
-------------------------

## 3. Generation

In [14]:
prompt_template = """사용자는 법률 문서와 관련된 질문을 하고 있습니다. \
검색된 정보를 기반으로, 질문에 **간결하게** 답변해 주세요.

RETRIEVED INFORMATION:
{retrieved_documents}

USER QUESTION:
{user_query}"""


def format_docs(hits):
    """검색된 문서들을 하나의 문자열로 합치기"""
    doc_list = [hit.payload["content"] for hit in hits]
    return "\n\n---\n\n".join(doc_list)


# 최종 프롬프트 확인
final_prompt = prompt_template.format(
    retrieved_documents=format_docs(hits),
    user_query=user_query,
)
print("[ 생성된 Prompt 미리보기 ]")
print(final_prompt[:300], "...")

[ 생성된 Prompt 미리보기 ]
사용자는 법률 문서와 관련된 질문을 하고 있습니다. 검색된 정보를 기반으로, 질문에 **간결하게** 답변해 주세요.

RETRIEVED INFORMATION:
제3조(전자서명의 효력) ① 전자서명은 전자적 형태라는 이유만으로 서명, 서명날인 또는 기명날인으로서의 효력이 부
인되지 아니한다.
② 법령의 규정 또는 당사자 간의 약정에 따라 서명, 서명날인 또는 기명날인의 방식으로 전자서명을 선택한 경우
그 전자서명은 서명, 서명날인 또는 기명날인으로서의 효력을 가진다.
제4조(전자서명의 발전을 위한 시책 수립) 정부는 전자서명의 안전성 ...


In [15]:
client = OpenAI(
    api_key=api_key,
    base_url="https://api.upstage.ai/v1",
)

# 답변 생성
stream = client.chat.completions.create(
    model="solar-pro2",
    messages=[{"role": "user", "content": final_prompt}],
    reasoning_effort="high",  # ⭐️ 추론 모드
    stream=True,
    temperature=0.6,
    top_p=0.9,
    max_tokens=1024,
    # 이 외에도 다른 파라미터를 설정할 수 있습니다
)

# stream 출력
for chunk in stream:
    response = chunk.choices[0].delta.content
    if response is not None:
        print(response, end="")

전자서명의 발전을 위한 시책은 다음과 같습니다(제4조 참조):  
1. 전자서명 신뢰성 제고 및 수단 다양성 확보  
2. 전자서명 제도 개선 및 관계 법령 정비  
3. 가입자·이용자 권익 보호  
4. 전자서명 상호연동 촉진  
5. 기술개발·표준화·인력 양성  
6. 안전한 암호 사용을 통한 신뢰성 확보  
7. 외국 전자서명 상호인정 등 국제협력  
8. 공공서비스 전자서명 안전 관리  
9. 그 밖의 전자서명 발전을 위한 필요 사항  

이 시책은 정부가 수립·시행하며, 전자서명의 안전성과 이용 활성화를 목표로 합니다.